In [131]:
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm

# assert len(tf.config.list_physical_devices('GPU')) > 0

# Taylor Swift lyrics dataset courtesy of https://www.kaggle.com/PromptCloudHQ/taylor-swift-song-lyrics-from-all-the-albums

In [98]:
text_file = open("lyrics.txt", "r")
all_lyrics = text_file.read()
print(all_lyrics)
print(len(all_lyrics))
text_file.close()

He said the way my blue eyes shined
Put those Georgia stars to shame that night
I said, "That's a lie"
Just a boy in a Chevy truck
That had a tendency of gettin' stuck
On backroads at night
And I was right there beside him all summer long
And then the time we woke up to find that summer gone
But when you think Tim McGraw
I hope you think my favorite song
The one we danced to all night long
The moon like a spotlight on the lake
When you think happiness
I hope you think that little black dress
Think of my head on your chest
And my old faded blue jeans
When you think Tim McGraw
I hope you think of me
September saw a month of tears
And thankin' God that you weren't here
To see me like that
But in a box beneath my bed
Is a letter that you never read
From three summers back
It's hard not to find it all a little bittersweet
And lookin' back on all of that, it's nice to believe
When you think Tim McGraw
I hope you think my favorite song
The one we danced to all night long
The moon like a spotl

In [99]:
all_lyrics = all_lyrics.replace("\n"," ").lower()
print(all_lyrics)

he said the way my blue eyes shined put those georgia stars to shame that night i said, "that's a lie" just a boy in a chevy truck that had a tendency of gettin' stuck on backroads at night and i was right there beside him all summer long and then the time we woke up to find that summer gone but when you think tim mcgraw i hope you think my favorite song the one we danced to all night long the moon like a spotlight on the lake when you think happiness i hope you think that little black dress think of my head on your chest and my old faded blue jeans when you think tim mcgraw i hope you think of me september saw a month of tears and thankin' god that you weren't here to see me like that but in a box beneath my bed is a letter that you never read from three summers back it's hard not to find it all a little bittersweet and lookin' back on all of that, it's nice to believe when you think tim mcgraw i hope you think my favorite song the one we danced to all night long the moon like a spotl

In [100]:
all_lyrics = all_lyrics.split(' ')
print(all_lyrics)

['he', 'said', 'the', 'way', 'my', 'blue', 'eyes', 'shined', 'put', 'those', 'georgia', 'stars', 'to', 'shame', 'that', 'night', 'i', 'said,', '"that\'s', 'a', 'lie"', 'just', 'a', 'boy', 'in', 'a', 'chevy', 'truck', 'that', 'had', 'a', 'tendency', 'of', "gettin'", 'stuck', 'on', 'backroads', 'at', 'night', 'and', 'i', 'was', 'right', 'there', 'beside', 'him', 'all', 'summer', 'long', 'and', 'then', 'the', 'time', 'we', 'woke', 'up', 'to', 'find', 'that', 'summer', 'gone', 'but', 'when', 'you', 'think', 'tim', 'mcgraw', 'i', 'hope', 'you', 'think', 'my', 'favorite', 'song', 'the', 'one', 'we', 'danced', 'to', 'all', 'night', 'long', 'the', 'moon', 'like', 'a', 'spotlight', 'on', 'the', 'lake', 'when', 'you', 'think', 'happiness', 'i', 'hope', 'you', 'think', 'that', 'little', 'black', 'dress', 'think', 'of', 'my', 'head', 'on', 'your', 'chest', 'and', 'my', 'old', 'faded', 'blue', 'jeans', 'when', 'you', 'think', 'tim', 'mcgraw', 'i', 'hope', 'you', 'think', 'of', 'me', 'september', 's

In [101]:
useless_characters = ["(", ")", "\"", "", ":", ",", ".", "!", "?", "\“", "\…", "<u+203d>"]
clean_lyrics = all_lyrics
for character in useless_characters:
    clean_lyrics = [word.replace(character,"") for word in clean_lyrics]

clean_lyrics = [word.replace("&amp;","and") for word in clean_lyrics]

regex = r"in'$"
clean_lyrics = [re.sub(regex, "ing", word) for word in clean_lyrics]

clean_lyrics = [re.sub("\'", "", word) for word in clean_lyrics]

print(clean_lyrics)

['he', 'said', 'the', 'way', 'my', 'blue', 'eyes', 'shined', 'put', 'those', 'georgia', 'stars', 'to', 'shame', 'that', 'night', 'i', 'said', 'thats', 'a', 'lie', 'just', 'a', 'boy', 'in', 'a', 'chevy', 'truck', 'that', 'had', 'a', 'tendency', 'of', 'getting', 'stuck', 'on', 'backroads', 'at', 'night', 'and', 'i', 'was', 'right', 'there', 'beside', 'him', 'all', 'summer', 'long', 'and', 'then', 'the', 'time', 'we', 'woke', 'up', 'to', 'find', 'that', 'summer', 'gone', 'but', 'when', 'you', 'think', 'tim', 'mcgraw', 'i', 'hope', 'you', 'think', 'my', 'favorite', 'song', 'the', 'one', 'we', 'danced', 'to', 'all', 'night', 'long', 'the', 'moon', 'like', 'a', 'spotlight', 'on', 'the', 'lake', 'when', 'you', 'think', 'happiness', 'i', 'hope', 'you', 'think', 'that', 'little', 'black', 'dress', 'think', 'of', 'my', 'head', 'on', 'your', 'chest', 'and', 'my', 'old', 'faded', 'blue', 'jeans', 'when', 'you', 'think', 'tim', 'mcgraw', 'i', 'hope', 'you', 'think', 'of', 'me', 'september', 'saw', 

In [102]:
# Find all unique characters in the joined string
vocab = sorted(set(clean_lyrics))
print(vocab)
print("There are", len(vocab), "unique words in the lyrics")

['', '1', '14', '158', '17', '18', '2', '200', '22', '230', '2am', '3', '32', '4', '45', '4th', '9th', 'a', 'a-side', 'a-team', 'abigail', 'about', 'about—', 'above', 'absent', 'absurd', 'accused', 'ace', 'achilles', 'aching', 'acing', 'across', 'act', 'acted', 'actress', 'actually', 'add', 'admit', 'adore', 'affair', 'afraid', 'after', 'again', 'against', 'age', 'aglow', 'ago', 'ah', 'ah-aah', 'ah-aah-aah-aah-aah-haa', 'ahead', 'ahh-ooh', 'aint', 'air', 'airplanes', 'aisle', 'album', 'aligned', 'alive', 'all', 'almost', 'alone', 'along', 'already', 'alright', 'always', 'am', 'ambition', 'amen', 'american', 'amount', 'an', 'and', 'angel', 'another', 'answer', 'anthem', 'anticipation', 'any', 'anybody', 'anymore', 'anyone', 'anything', 'anyway', 'anywhere', 'apart', 'apartment', 'apartnow', 'apologies', 'apology', 'applause', 'are', 'arent', 'argue', 'arm', 'armor', 'arms', 'around', 'as', 'aside', 'ask', 'asked', 'asking', 'asleep', 'at', 'attached', 'attitude', 'autumn', 'avalanche', 

In [104]:
# Enumerate the vocab (assign each word a number) to create a mapping of words - numbers
word2idx = {u:i for i, u in enumerate(vocab)}

In [107]:
# Reverse the mapping
idx2word = np.array(vocab)

In [110]:
# Vectorize the lyrics
vectorized_lyrics = []
for word in clean_lyrics:
    vectorized_lyrics.append(word2idx[word])
vectorized_lyrics = np.array(vectorized_lyrics)

print(vectorized_lyrics)

[ 905 1679 2030 ... 2082 2362    0]


In [111]:
def get_batch(vectorized_lyrics, seq_length, batch_size):
  # the length of the vectorized songs string
  n = vectorized_lyrics.shape[0] - 1
  # randomly choose the starting indices for the examples in the training batch
  idx = np.random.choice(n-seq_length, batch_size)

  # construct a list of input sequences for the training batch
  input_batch = [vectorized_lyrics[i:i+seq_length] for i in idx]
  # construct a list of output sequences for the training batch
  output_batch = [vectorized_lyrics[i+1:i+1+seq_length] for i in idx]

  # x_batch, y_batch provide the true inputs and targets for network training
  x_batch = np.reshape(input_batch, [batch_size, seq_length])
  y_batch = np.reshape(output_batch, [batch_size, seq_length])

  return x_batch, y_batch

In [114]:
# Demonstrate the batching over the timesteps
x_batch, y_batch = get_batch(vectorized_lyrics, seq_length=5, batch_size=1)
print(x_batch)
print(y_batch)
for i, (input_idx, target_idx) in enumerate(zip(np.squeeze(x_batch), np.squeeze(y_batch))):
    print("Step {:3d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2word[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2word[target_idx])))

[[2082 2030  419 1787  614]]
[[2030  419 1787  614  989]]
Step   0
  input: 2082 ('to')
  expected output: 2030 ('the')
Step   1
  input: 2030 ('the')
  expected output: 419 ('crickets')
Step   2
  input: 419 ('crickets')
  expected output: 1787 ('sing')
Step   3
  input: 1787 ('sing')
  expected output: 614 ('everything')
Step   4
  input: 614 ('everything')
  expected output: 989 ('i')


In [115]:
def LSTM(rnn_units): 
  return tf.keras.layers.LSTM(
    rnn_units, 
    return_sequences=True, 
    recurrent_initializer='glorot_uniform',
    recurrent_activation='sigmoid',
    stateful=True,
  )

In [116]:
### Defining the RNN Model ###

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    # Layer 1: Embedding layer to transform indices into dense vectors 
    #   of a fixed embedding size
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),

    # Layer 2: LSTM with `rnn_units` number of units. 
    LSTM(rnn_units),

    # Layer 3: Dense (fully-connected) layer that transforms the LSTM output
    #   into the vocabulary size. 
    tf.keras.layers.Dense(vocab_size)
  ])

  return model

# Build a simple model with default hyperparameters. You will get the 
#   chance to change these later.
model = build_model(len(vocab), embedding_dim=256, rnn_units=1024, batch_size=32)

In [117]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (32, None, 256)           608000    
_________________________________________________________________
lstm (LSTM)                  (32, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (32, None, 2375)          2434375   
Total params: 8,289,351
Trainable params: 8,289,351
Non-trainable params: 0
_________________________________________________________________


In [118]:
x, y = get_batch(vectorized_lyrics, seq_length=100, batch_size=32)
pred = model(x)
print("Input shape:      ", x.shape, " # (batch_size, sequence_length)")
print("Prediction shape: ", pred.shape, "# (batch_size, sequence_length, vocab_size)")

Input shape:       (32, 100)  # (batch_size, sequence_length)
Prediction shape:  (32, 100, 2375) # (batch_size, sequence_length, vocab_size)


In [119]:
# obtain predictions from untrained model
sampled_indices = tf.random.categorical(pred[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([1726, 2142,  369, 1813,  718,  211,  938, 1988, 1627, 1463, 2198,
       1262, 2052, 1880,  127, 2155,  921, 1953, 1953, 1138, 1419,  232,
       2152, 1982, 2064,  468, 1033,   63,  675,  733, 1598, 2160, 2066,
        301, 1493, 1605,   58,  733,  576, 1137,  703, 1981, 1389, 1479,
       1279, 1531, 1079,  131,  201, 2032, 1282, 1837, 1448, 2109, 1230,
       2345,  963, 1375,  556, 1576,  577,  272,   40, 2107, 1627,  762,
         92, 1478, 1974,  422,  225, 1414, 1087,  510, 1928,  321, 1651,
       1271, 1064,  230, 1339, 1467,  689, 1617,  418,  402,  333,   60,
       1812, 1495, 1210, 1187, 2114, 2014, 1124, 1303,  470,  391, 1453,
        260])

In [123]:
# Decode predictions from untrained model, find they're a bit rubbish
print(x)
print("Input: \n", repr(" ".join(idx2word[x[0]])))
print()
print("Next Word Predictions: \n", repr(" ".join(idx2word[sampled_indices])))

[[2244 2261 1796 ... 2229 1633 1759]
 [2027  616  989 ...  989  172 2368]
 [ 936  265 2362 ... 2109 1357 2053]
 ...
 [1368 1018 1746 ... 2229  593 2082]
 [2030 1891 2260 ... 1610 2053  672]
 [  47   72  989 ... 1305 1602 1147]]
Input: 
 'we were sitting at our favorite spot in town and you looked at me got down on one knee take me back to the time when we walked down the aisle our whole town came and our mamas cried you said i do and i did too take me home where we met so many years before well rock our babies on that very front porch after all this time you and i ill be eighty-seven; youll be eighty-nine ill still look at you like the stars that shine in the sky oh my my my i was riding shotgun'

Next Char Predictions: 
 'septembers tuck cold sleeve flew born hero tails revolved penny village mirror thirteen sprung ball twist heat style style lighthearted paces breathe twice switching throwing darling its already feet fond recognize typical ticking case planning reflects alive fond ea

In [124]:
### TRAINING THE MODEL: Part 1: Defining the loss function ###

# define the loss function to compute and return the loss between the true labels and predictions (logits). 
# Set the argument from_logits=True.
def compute_loss(labels, logits):
  loss = tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
  return loss

# compute the loss using the true next characters from the example batch 
# and the predictions from the untrained model several cells above
example_batch_loss = compute_loss(y, pred)

print("Prediction shape: ", pred.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (32, 100, 2375)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       7.7727013


In [126]:
### Hyperparameter setting and optimization ###

# Optimization parameters:
num_training_iterations = 2000  # Increase this to train longer
batch_size = 4  # Experiment between 1 and 64
seq_length = 100  # Experiment between 50 and 500
learning_rate = 5e-3  # Experiment between 1e-5 and 1e-1

# Model parameters: 
vocab_size = len(vocab)
embedding_dim = 256 
rnn_units = 1024  # Experiment between 1 and 2048

# Checkpoint location: 
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "my_ckpt")

In [137]:
### Define optimizer and training operation ###

# instantiate a new model for training using the `build_model`
# function and the hyperparameters created above.'''
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)

# TODO: instantiate an optimizer with its learning rate.
#   Checkout the tensorflow website for a list of supported optimizers.
#   https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/
#   Try using the Adam optimizer to start
optimizer = tf.keras.optimizers.Adam(learning_rate)


@tf.function
def train_step(x, y): 
  # Use tf.GradientTape()
  with tf.GradientTape() as tape:
  
    # Feed the current input into the model and generate predictions
    y_hat = model(x)
  
    # compute the loss
    loss = compute_loss(y, y_hat)

  # Now, compute the gradients 
#    complete the function call for gradient computation. 
#       Remember that we want the gradient of the loss with respect all 
#       of the model parameters. 
#       HINT: use `model.trainable_variables` to get a list of all model
#       parameters.
  grads = tape.gradient(loss, model.trainable_variables)
  
  # Apply the gradients to the optimizer so it can update the model accordingly
  optimizer.apply_gradients(zip(grads, model.trainable_variables))
  return loss

##################
# Begin training!#
##################

history = []
if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

pbar = tqdm(range(num_training_iterations))
for iter in pbar:

  # Grab a batch and propagate it through the network
  x_batch, y_batch = get_batch(vectorized_lyrics, seq_length, batch_size)
  loss = train_step(x_batch, y_batch)

  # Update the progress bar
  history.append(loss.numpy().mean())
  pbar.set_description("loss: {}".format(loss.numpy().mean()))

  # Update the model with the changed weights!
  if iter % 100 == 0:     
    model.save_weights(checkpoint_prefix)
    
    
# Save the trained model and the weights
model.save_weights(checkpoint_prefix)

loss: 5.160910129547119:   4%|▍         | 88/2000 [01:24<30:25,  1.05it/s]  


KeyboardInterrupt: 